In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.head()

In [ ]:
df = dataset
df['sex'] = pd.factorize(df['sex'])[0]
df['smoker'] = pd.factorize(df['smoker'])[0]
df['region'] = pd.factorize(df['region'])[0]
df['smoker'] = pd.factorize(df['smoker'])[0]
dataset = df
print(dataset.head())
test_dataset = dataset.sample(frac=0.2)
test_labels = test_dataset.pop('expenses')
print(len(test_dataset))
train_dataset = dataset[~dataset.isin(test_dataset)].dropna()
train_labels = train_dataset.pop('expenses')
print(train_dataset.head())

In [ ]:
normalizer = layers.Normalization()
normalizer.adapt(np.array(train_dataset))
model = Sequential([normalizer,layers.Dense(16, activation='relu'),layers.Dense(4, activation='relu'),layers.Dropout(0.2),layers.Dense(1)])
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1),loss='mae',metrics=['mae', 'mse'])
model.build(input_shape=(None, train_dataset.shape[1]))
model.summary()

In [ ]:
history = model.fit(train_dataset,train_labels,epochs=100,validation_split=0.5,verbose=0,)
print(history)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
